# Digit Classification with sklearn

By: Jacobo Fernandez-Vargas and Luca Citi

Based on the [scikit-learn documentation](http://scikit-learn.org/stable/tutorial/basic/tutorial.html)

## Loading the 'digits' dataset

Scikit-learn comes with a few standard datasets, for instance the *Iris* and *digits* datasets for classification and the *Boston house prices* dataset for regression.

In [ ]:
from sklearn import datasets

digits = datasets.load_digits()

A dataset is a dictionary-like object that holds all the data and some metadata about the data. The data is stored in the `.data` member, which is usually a `n_samples` by `n_features` array. In the case of supervised problem, one or more response variables are stored in the `.target` member.

For instance, in the case of the digits dataset, `digits.data` gives access to the features that can be used to classify the digits samples:

In [ ]:
print(digits.data.shape)
print(digits.data)

and `digits.target` gives the ground truth for the digit dataset, that is the number corresponding to each digit image that we are trying to learn:

In [ ]:
print(digits.target.shape)
print(digits.target)

We can view the individual images in this dataset.  Here we'll just display the first 4.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# display the first 4 images in the dataset
for i in range(4):
    plt.matshow(digits.images[i], cmap='binary')

## Learning and predicting
In the case of the digits dataset, the task is to predict, given an image, which digit it represents. We are given samples of each of the 10 possible classes (the digits 0 through 9). These can be used to fit an estimator to predict the class an unseen example belongs to.

In scikit-learn, an estimator for classification is a Python object that implements the methods `fit(X, y)` and `predict(T)`. If you remember in the last lab we saw that the preprocessing classes had the methods `fit(X, y)` and `transform(T)`. We will see the differences later.

An example of an estimator is the class `sklearn.svm.SVC` that implements support vector classification. The constructor of an estimator takes as arguments the parameters of the model, but for the time being, we will consider the estimator as a black box:

In [ ]:
from sklearn import svm

clf = svm.SVC(gamma=0.0001, C=100.)

We call our estimator instance `clf`, as it is a classifier. **It now must be fitted to the model, that is, it must learn from the data**. This is done by passing our training set to the `fit` method. As a training set, let us use all the images of our dataset apart from the last one. We select this training set with the `[:-1]` Python syntax, which produces a new array that contains **all but the last entry** of `digits.data`:

In [ ]:
clf.fit(digits.data[:-1], digits.target[:-1])

Now you can predict new values. In particular, we can ask to the classifier to recognise the digit of our last image in the digits dataset, which we have not used to train the classifier:

In [ ]:
clf.predict(digits.data[-1:])

The corresponding image is the following.
As you can see, it is a challenging task: the images are of poor resolution.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

plt.matshow(digits.images[-1], cmap='binary')
print("Correct label",digits.target[-1], "SVM prediction",clf.predict(digits.data[-1:]))

Did your SVM classify the digit correctly?

Was this example which it classified used in the training set?

## Grid-search

Almost all classifiers have parameters that need to be set.  For example, the SVM above which you used relied on "C" and "gamma" being chosen carefully to work well.  These are called "hyper parameters" of the ML algorithm.

The meaning and roles of these kinds of pararmeters differ largely between classifiers and their appropiate selection can have huge impact on the results (but some times they may have almost no effect). This parameter fitting should happen inside a cross-validation process. Scikit-learn provides `GridSeachCV` that, given data, fit of an estimator on each combination of parameters on parameter grid and then chooses the parameters to maximize the cross validation score. To use this, we need to specify the classifier that we want to use and the values of the parameters that we want to test (through a dictionary). Interestingly, the  `GridSeachCV` constructor creates an object that can be used as a predcictor, using its `fit`, `predict`, `score`, ... methods; in other words it is a meta-predictor (i.e. a preditor built from other predictors, an SVM in thisd case) that can be used like any other predictors:

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = svm.SVC(gamma=0.01, C=10.)
Cs = np.logspace(-1, 3, 5)  # five logarithmically spaced values of C
Gs = np.logspace(-7, -0, 5)  # five logarithmically spaced values of gamma
clf = GridSearchCV(estimator=clf, param_grid=dict(C=Cs, gamma=Gs), n_jobs=-1)

clf.fit(digits.data, digits.target)
clf.score(digits.data, digits.target)

**IMPORTANT NOTE**

The way we trained and tested (fit, score) is WRONG! You should not calculate the score with the same data used for training, as we saw in the previous lectures and lab. We did it this way for demonstration purposes of how the GridSeachCV class works. The fact that we see a score of 1.0 (100%) should make us suspicious: it's often a sign that we did something wrong (either that or the problem is trivial).

We can see how the different values of C and gamma affect the performance:

In [ ]:
# Grid results for last fold
scores = clf.cv_results_['mean_test_score'].reshape(len(Cs), len(Gs))
extent = np.log10([Gs[0], Gs[-1], Cs[0], Cs[-1]])
im = plt.imshow(scores, extent=extent, origin='lower')
plt.colorbar(im)
plt.contour(np.log10(Gs), np.log10(Cs), scores)
plt.xlabel('log10(Gamma)')
plt.ylabel('log10(C)')
print('Best parameters: ', clf.best_params_)

## Automated cross validation

In the previous lab we saw the importance of using fold validation and we saw a way of implementing it. Fortunately, sklearn provides us with tools that take care of most of the heavy lifting: `cross_val_score` and `cross_val_predict`. The former returns only the score (the error function value, for example MSE), while the latter returns all the predicted values. In general, the score is enough to validate a model and compare different options. However, having all the predicted values may be useful in some cases.

The following code may take some time as it is performing nested cross-validation. This is because the `cross_val_score` function performs cross-validation on a meta-predictor that internally performs another cross-validation loop. The result is a nested cross-validation where `cross_val_score` performs the outer loop for performance estimation and the `GridSeachCV` object takes care of the inner one for parameter tuning.

In [ ]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(clf, digits.data, digits.target, cv=5) 
print(score)

As you can see the score from the fold validation varies fold to fold and it is not 1.

# Testing different classifiers

### Exercise 1

Try other classifiers to solve the problem and compare their results. Suggestions for classifiers: Logistic Regression, Decision Trees. You will need to look up the documentation to find which are the parameters to be fit as well as the different classes available. Once you do this, you will see how easy is to change the model and test different approaches with this set up.

# Regression and unsupervised learning

In this lab we have seen how to use sklearn to solve a classification problem, which is why we used support vector machines. However, everything that we have seen here can be applied to both regression and (to some extent) unsupervised learning. Depending on the problem we will need to vary the model used for the prediction but most of the concepts seen on this lab also apply to those cases.

### Exercise 2

Load the boston dataset form sklearn `datasets.load_boston` and use what you have learned to solve a regression problem.